<h1>ДЗ по эконометрике!</h1>
<h2>Скворцов Иван, Анна Пономаренко, БЭК-181</h2>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Временной ряд с месячной частотой

## Взять языка



Временной ряд взят [отсюда](https://www.kaggle.com/piyushagni5/monthly-sales-of-french-champagne).

In [ ]:
# считаем файлы
ts = pd.read_csv('monthly_champagne_sales.csv')

In [ ]:
ts['Month'] = pd.to_datetime(ts['Month'], format = '%Y-%m')

In [ ]:
ts.set_index('Month', inplace = True)

In [ ]:
ts.index = ts.index.to_period("M") # говорим что индекс у нас с месячной частотой

## Намалевич

Сначала сам ряд. Просто и лаконично. И легко дышится.

In [ ]:
fig, ax = plt.subplots(1, figsize = (20, 10))

ts.plot(ax = ax)
ax.set(title = 'Champagne sales per month', xlabel = 'Date', ylabel = 'Sales')

plt.show()

Теперь автокорреляцию. Схватим очевидную сезонность.

In [ ]:
from statsmodels.graphics import tsaplots

fig = tsaplots.plot_acf(ts["Sales"], lags=20)
plt.show()

Ну и наконец построим ряд с наложением.

In [ ]:
fig, ax = plt.subplots(1, figsize = (20, 10))

ax.plot(ts.index.month, ts.Sales, 'o', color = 'lightcoral', alpha = .8)
ax.set_xticks([i for i in range(1, 13)])

plt.show()

## Твиттер

Выбранный ряд явно сезонный: это видно на всех графиках, которые мы привели ☺️. Тренд если и наблюдается, то довольно незначительный; при этом ряд не стационарен: разброс его значений непостоянен. Логарифм брать не будем, поскольку это ничего особо не меняет. 

## Двенадцать месяцев

Трейнтестим!

In [ ]:
ts_train = ts.iloc[0:int(0.89 * ts.shape[0]), :]
ts_test = ts.iloc[int(0.89 * ts.shape[0]):, :]

print(f'Месяцев в тестовой выборке: {ts_test.shape[0]}')

In [ ]:
fig, ax = plt.subplots(1, figsize = (20, 10))

ts_train.plot(ax = ax)
ts_test.plot(ax = ax, color = 'orange')
ax.set(title = 'Champagne sales per month', xlabel = 'Date', ylabel = 'Sales')

plt.show()

In [ ]:
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.arima import ARIMA, AutoARIMA
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.ets import AutoETS
from sktime.performance_metrics.forecasting import MeanAbsoluteError

mae = MeanAbsoluteError() # y_true, y_pred

### Наивная

In [ ]:
nfc = NaiveForecaster(strategy='last')
nfc.fit(ts_train['Sales'])

In [ ]:
fh = ForecastingHorizon(ts_test.index, is_relative=False)

In [ ]:
def plot_fc(y_train, y_test, fc):

  err = np.round(mae(y_test, fc), 2)

  fig, ax = plt.subplots(1, figsize = (20, 10))

  y_train.plot(ax = ax, color = 'cornflowerblue', label = 'Train', linewidth = 1.7)
  y_test.plot(ax = ax, color = 'sandybrown', label = 'Test', linewidth = 1.7)
  fc.plot(ax = ax, color = 'mediumseagreen', linestyle = 'dashed', label = 'Forecast', linewidth = 1.7)

  ax.set(title = '', xlabel = 'Date', ylabel = 'Value')
  ax.text(0.013, 0.95, f'MAE: {err}', fontsize = 24, horizontalalignment='left', verticalalignment='center', transform = ax.transAxes)

  plt.legend()
  plt.show()

In [ ]:
plot_fc(ts_train['Sales'], ts_test['Sales'], nfc.predict(fh))

### Сезонная наивная

In [ ]:
nfc = NaiveForecaster(strategy='last', sp = 12)
nfc.fit(ts_train['Sales'])
fh = ForecastingHorizon(ts_test.index, is_relative=False)

In [ ]:
plot_fc(ts_train['Sales'], ts_test['Sales'], nfc.predict(fh))

### SARIMA(1, 1, 1)(1, 0, 0)[12]

In [ ]:
arima = ARIMA(order = (1, 1, 1), seasonal_order = (1, 0, 0, 12), suppress_warnings = True)
arima.fit(ts_train['Sales'])
fh = ForecastingHorizon(ts_test.index, is_relative=False)

In [ ]:
plot_fc(ts_train['Sales'], ts_test['Sales'], arima.predict(fh))

### Auto-ARIMA

In [ ]:
arima = AutoARIMA(sp = 12, suppress_warnings = True, information_criterion = 'aic', scoring = 'mae')
arima.fit(ts_train['Sales'])
fh = ForecastingHorizon(ts_test.index, is_relative=False)

In [ ]:
plot_fc(ts_train['Sales'], ts_test['Sales'], arima.predict(fh))

### ETS(AAA)

In [ ]:
ets = ExponentialSmoothing(trend = 'add', seasonal = 'add', sp = 12)
ets.fit(ts_train['Sales'])
fh = ForecastingHorizon(ts_test.index, is_relative=False)

In [ ]:
plot_fc(ts_train['Sales'], ts_test['Sales'], ets.predict(fh))

### AutoETS

In [ ]:
ets = AutoETS(auto = True, sp = 12)
ets.fit(ts_train['Sales'].astype(np.float64))
fh = ForecastingHorizon(ts_test.index, is_relative=False)

In [ ]:
plot_fc(ts_train['Sales'], ts_test['Sales'], ets.predict(fh))

## Кто на свете всех милее?

Лучшая модель &mdash; ???

In [ ]:
ets = ExponentialSmoothing(trend = 'add', seasonal = 'add', sp = 12)
ets.fit(ts['Sales'])
fh = ForecastingHorizon([i for i in range(1, 13)], is_relative=True)

In [ ]:
fig, ax = plt.subplots(1, figsize = (20, 10))

ts['Sales'].plot(ax = ax, color = 'cornflowerblue', label = 'Train', linewidth = 1.7)
ets.predict(fh).plot(ax = ax, color = 'mediumseagreen', linestyle = 'dashed', label = 'Forecast', linewidth = 1.7)
ax.set(title = '', xlabel = 'Date', ylabel = 'Value')

plt.legend()
plt.show()

# Панельные данные

## Хождение в народ

- J60 &mdash; совокупный доход
- J69_9C &mdash; год рождения




In [ ]:
panel = pd.read_stata('mydata2.dta').drop(columns = ['ID_W'])

In [ ]:
panel = panel[~panel.J60.isin(['ЗАТРУДНЯЮСЬ ОТВЕТИТЬ', 'ОТКАЗ ОТ ОТВЕТА', 'НЕТ ОТВЕТА'])]
panel = panel[~panel.J69_9C.isin(['ЗАТРУДНЯЮСЬ ОТВЕТИТЬ', 'ОТКАЗ ОТ ОТВЕТА', 'НЕТ ОТВЕТА'])]

In [ ]:
panel.dropna(inplace=True)

In [ ]:
panel['J69_9C'] = panel['J69_9C'].astype('object').astype(int)
panel['J60'] = panel['J60'].astype('object').astype(np.float64)

In [ ]:
panel['age'] = 2019 - panel['J69_9C']

In [ ]:
panel['income'] = panel['J60']

In [ ]:
panel = panel[['idind', 'year', 'age', 'income']]

In [ ]:
panel['age2'] = np.square(panel['age'])
panel['lnincome'] = np.log1p(panel['income'])

In [ ]:
panel = panel[panel.income != 0]

## Кому на Руси жить хорошо?

Вопрос будет очень простой и очень бессмысленный: как влияет возраст на общий трудовой и нетрудовой доход? Чтобы не позориться, добавим еще квадрат возраста.

## Сделай красиво!

In [ ]:
fig, ax = plt.subplots(1, figsize = (20, 10))

ax.plot(panel.age, panel.lnincome, 'o', color = 'lightcoral', alpha = .7)
ax.set(title = 'Зависимость логарифма дохода индивида от возраста', xlabel = 'Возраст', ylabel = 'Доход')

plt.show() # очень красиво.

## Три сестры

In [ ]:
panel = panel.set_index(['idind', 'year'])

In [ ]:
from linearmodels.panel.model import PooledOLS, RandomEffects, PanelOLS

### Сквозная модель

In [ ]:
mod = PooledOLS(panel.lnincome, panel[['age', 'age2']])
mod = mod.fit()

In [ ]:
def plot_vs_hat(model, data):
    fig, ax = plt.subplots(1, figsize = (20, 10))

    ax.plot(data.lnincome, model.predict(data[['age', 'age2']]).predictions, 'o', color = 'lightcoral', alpha = .7)
    ax.set(title = 'Предсказания vs. реальные значения', xlabel = 'yhat', ylabel = 'y')

    plt.show()


In [ ]:
plot_vs_hat(mod, panel)

### FE-модель

In [ ]:
mod = PanelOLS(panel.lnincome, panel[['age', 'age2']])
print(mod.fit())

### RE-модель

In [ ]:
mod = RandomEffects(panel.lnincome, panel[['age', 'age2']])
print(mod.fit())

Как выбрать лучшую?